Importy potrzebnych biblitek:

In [40]:
import pandas as pd
import re
import numpy as np
import random
import copy
from numba import njit

Stałe oznaczające koszt transponderów oraz ich maksymalną ilość:

In [41]:
TRANSPONDERS = {
    10: 1,
    40: 3,
    100: 5
}
MAX_TRANSPONDERS = 96

Odczytywanie miast z ich położeniem:

In [56]:
with open('networks/NODES.txt', 'r') as file:
    data = file.read()


pattern = r'(\w+) \(\s*([\d.]+)\s*([\d.]+)\s*\)'

matches = re.findall(pattern, data)

nodes_df = pd.DataFrame(matches, columns=['City', 'Longitude', 'Latitude'])

nodes_df['Longitude'] = nodes_df['Longitude'].astype(float)
nodes_df['Latitude'] = nodes_df['Latitude'].astype(float)

#print(nodes_df)

Odczytywanie połączeń między miastami:

In [43]:
with open('networks/LINKS.txt', 'r') as file:
    data = file.read()

pattern = r'(\w+) \(\s*(\w+)\s*(\w+)\s*\)'

matches = re.findall(pattern, data)

links_df = pd.DataFrame(matches, columns=['ID', 'City_1', 'City_2'])

#print(links_dict)

Odczytywanie z plików zapotrzebowań:

In [44]:
with open('networks/DEMANDS.txt', 'r') as file:
    data = file.read()


pattern = r'(\w+) \(\s*(\w+)\s*(\w+)\s*\) (\d+) (\d+\.\d+) (\w+)'

matches = re.findall(pattern, data)

demands_df = pd.DataFrame(matches, columns=['ID', 'City_1', 'City_2', 'routing_unit', 'demand_value', 'max_route'])
demands_df = demands_df.drop(['routing_unit', 'max_route'], axis = 1)

demands_dict = dict(zip(demands_df['ID'], demands_df['demand_value'].astype(float)))

Odczytywanie z pliku ścieżek pomiędzy miastami:

In [45]:
def parse_file(file_content):
    dict_data = {}
    current_demand = None
    current_p = None

    demand_pattern = re.compile(r'Demand_(\d+_\d+)')
    p_pattern = re.compile(r'P_(\d+)\s*\(\s*(.*?)\s*\)')

    lines = file_content.split('\n')
    for line in lines:
        demand_match = demand_pattern.search(line)
        p_match = p_pattern.search(line)

        if demand_match:
            current_demand = 'Demand_' + demand_match.group(1)
            dict_data[current_demand] = {}
        elif p_match and current_demand:
            current_p = 'P_' + p_match.group(1)
            links = re.findall(r'Link_\d+_\d+', p_match.group(2))
            dict_data[current_demand][current_p] = links

    return dict_data

with open('networks/PATHS.txt', 'r') as file:
    file_content = file.read()

result_dict = parse_file(file_content)

Generowanie populacji początkowej:

In [46]:
def generate_genotype(paths_num = 7, min = 0, max = 3):
    demnand_num = demands_df.shape[0]
    trans_num = len(TRANSPONDERS)
    genotype = np.random.randint(min, max, (demnand_num, paths_num, trans_num))
    return genotype

def generate_population(n):
    population = []
    for _ in range(n):
        population.append(generate_genotype())
    return population


Funkcja służąca do oceny osobnika:

In [47]:
def genotype_cost(individual, pen_const = 2):
    links_dict = dict(zip(links_df['ID'], np.zeros(len(links_df), dtype=int)))
    temp_demands_dict = dict(zip(demands_df['ID'], np.zeros(len(demands_df), dtype=int)))
    transponders_cost = 0
    demand_num = 0
    for demand_key, demand_value in result_dict.items():
        path_num = 0
        demand_cost = 0
        for path_values in demand_value.values():
            demand_cost += individual[demand_num][path_num][0]*TRANSPONDERS[10] + \
                           individual[demand_num][path_num][1]*TRANSPONDERS[40] + \
                           individual[demand_num][path_num][2]*TRANSPONDERS[100]
            temp_demands_dict[demand_key] += individual[demand_num][path_num][0]*10 + \
                                             individual[demand_num][path_num][1]*40 + \
                                             individual[demand_num][path_num][2]*100
            for link in path_values:
                links_dict[link] += individual[demand_num][path_num][0] + \
                                    individual[demand_num][path_num][1] + \
                                    individual[demand_num][path_num][2]
                
            path_num += 1
        transponders_cost += demand_cost
        demand_num += 1

    trans_exceeding_penalty = 0
    for key, value in links_dict.items():
        if value > MAX_TRANSPONDERS:
            trans_exceeding_penalty += (value-MAX_TRANSPONDERS) ** pen_const + 200

    demand_not_complited = 0
    for key, value in temp_demands_dict.items():
        if value < demands_dict[key]:
            demand_not_complited += (demands_dict[key]-value) ** pen_const + 200

    return transponders_cost + trans_exceeding_penalty + demand_not_complited

Selekcja ruletkowa:

In [48]:
def gen_roulette(popu, out_pop = 1):
    out_pop *= len(popu)
    new_population = []
    sum = 0.0
    for individual in popu:
        sum += 1/individual[1]
    for _ in range(out_pop):
        value = random.uniform(0.0, sum)
        new_sum = 0.0
        for i in range(len(popu)):
            new_sum += 1/popu[i][1]
            if new_sum >= value:
                new_population.append(copy.deepcopy(popu[i]))
                break
    return new_population

Selekcja turniejowa:

In [49]:
def gen_tournament(popu, out_pop = 1, tournament_size=2):
    out_pop *= len(popu)
    new_population = []
    for _ in range(out_pop):
        selected_candidates = random.sample(range(len(popu)), tournament_size)
        winner = min(selected_candidates, key=lambda x: popu[x][1])
        new_population.append(copy.deepcopy(popu[winner]))
    return new_population

Algorytm krzyżowania jendopunktowego:

In [50]:
def single_point_crossover(parent1, parent2):
    crossover_point = np.random.randint(1, min(len(parent1), len(parent2)))

    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]), axis = 0)
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]), axis = 0)
    return child1, child2

Mutacja gaussowska:

In [51]:
@njit
def gauss_mutation(individual, mutation_power, mutation_percent):
    for i in range(len(individual)):
        if random.uniform(0, 100) < mutation_percent:
            individual[i] = abs(int(random.gauss(individual[i], mutation_power)))
            individual[i] = individual[i] % 3
    return individual

Mutacja binarna:

In [52]:
@njit
def binary_mutation(individual, mutation_power, mutation_percent):
    for i in range(len(individual)):
        if random.uniform(0, 100) < mutation_percent:
            individual[i] = np.random.randint(0,3)
    return individual

Funkcje do generowania podsumowania w pliku .txt:

In [53]:
def genotype_raport(individual):
    temp_demands_dict = dict(zip(demands_df['ID'], np.zeros(len(demands_df), dtype=int)))
    demand_num = 0
    for demand_key, demand_value in result_dict.items():
        path_num = 0
        demand_cost = 0
        for path_values in demand_value.values():
            temp_demands_dict[demand_key] += individual[demand_num][path_num][0]*10 + individual[demand_num][path_num][1]*40 + individual[demand_num][path_num][2]*100
            path_num += 1
        demand_num += 1

    for key, value in temp_demands_dict.items():
        temp_demands_dict[key] = [value, demands_dict[key]]

    return temp_demands_dict

def genotype_to_file(individual, file_name = 'model.txt'):
    with open(file_name, 'w') as file:
        file.write(f'\n\n\n\n\nGenotype value is {individual[1]}\n')
        demand_num = 0
        raport_dict = genotype_raport(individual[0])
        for key, value in raport_dict.items():
            file.write(f'{key}    {value[0]}   {value[1]}     diff: {value[0]-value[1]}\n')
        for demand_key, demand_value in result_dict.items():
            path_num = 0
            file.write(f'\n\n\n\n\n{demand_key}:\n')
            for path_key, path_values in demand_value.items():
                file.write(f'{path_key}: 10 -> {individual[0][demand_num][path_num][0]}    40 -> {individual[0][demand_num][path_num][1]}   100 -> {individual[0][demand_num][path_num][2]}\n')
                path_num += 1
            demand_num += 1

Główna pętla algorytmu ewolucyjnego:

In [55]:
def evolution(population_size = 500, crossover_percent=10, selection=gen_tournament, crossover=single_point_crossover,
              mutation_power = 0.5, mutation_percent=10, mutation=gauss_mutation, iterations=1000, elite_size_percent = 10):
    population = generate_population(population_size)
    values = []
    for individual in population:
        values.append(genotype_cost(individual))
    rated_population = [list(pair) for pair in zip(population, values)]

    shape = population[0].shape
    elite = population_size * elite_size_percent // 100
    non_elite = population_size - elite

    for iteration in range(iterations):
        #selekcja
        new_population = selection(rated_population)

        for i in range(len(new_population)):
            new_population[i][0] = np.ravel(new_population[i][0])

        #krzyżowanie
        crossovers = population_size * crossover_percent // 100
        for _ in range(crossovers):
            i1 = random.randint(0, population_size-1)
            i2 = random.randint(0, population_size-1)
            new_population[i1][0], new_population[i2][0] = crossover(new_population[i1][0], new_population[i2][0])

        #mutacja
        for i in range(len(new_population)):
            new_population[i][0] = mutation(new_population[i][0], mutation_power, mutation_percent)
        
        for i in range(len(new_population)):
            new_population[i][0] = np.reshape(new_population[i][0], shape)
            new_population[i][1] = genotype_cost(new_population[i][0])
        

        rated_population = sorted(rated_population, key=lambda x: x[1])[:elite] + sorted(new_population, key=lambda x: x[1])[:non_elite]

        best_individual = min(rated_population, key=lambda x: x[1])
        
        if iteration % 50 == 0:
            print(f"Best individual so far: {best_individual[1]}")
            print(iteration)
            genotype_to_file(best_individual)
        if 0 < iteration < 301 and iteration % 50 == 0:
            mutation_percent -= 1
            print(f'Mutation percent was decreased: {mutation_percent}')
evolution()


Best individual so far: 1238197
0
Best individual so far: 4301.0
50
Mutation percent was decreased: 9
Best individual so far: 1123.0
100
Mutation percent was decreased: 8
Best individual so far: 873
150
Mutation percent was decreased: 7
Best individual so far: 825
200
Mutation percent was decreased: 6
Best individual so far: 781
250
Mutation percent was decreased: 5
Best individual so far: 739
300
Mutation percent was decreased: 4
Best individual so far: 706
350
Best individual so far: 691
400
Best individual so far: 674
450
Best individual so far: 669
500
Best individual so far: 658
550
Best individual so far: 657
600
Best individual so far: 649
650
Best individual so far: 644
700
Best individual so far: 643
750
Best individual so far: 638
800
Best individual so far: 634
850
Best individual so far: 634
900
Best individual so far: 633
950
